In [1]:
import json, urllib

In [5]:
import csv
import locale
import collections

#DrinkInfo = collections.namedtuple('DrinkInfo', 'name ingredients instructions src')

locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 

drinkByName = dict()

with open('mr-boston-flattened.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    line_count = 0
    
    for row in reader:
        line_count += 1
        if line_count == 1:
            print(row)
            
        else:
            
            name = row[0]
            
            ingredients = list()
            for i in range(0, 6):
                ing = (row[3+i*2], row[2+i*2])
                ingredients.append(ing)
            
            drink = dict()
            drink['name'] = name
            drink['ingredients'] = ingredients
            drink['instructions'] = row[14]
            
            drink['reviews'] = []
            drink['rating'] = -1
            
            #drink = DrinkInfo(name=name, ingredients=ingredients, instructions=row[14], #glass=(row[15], row[16]),
            #                 src = "http://worldartsme.com/images/cocktail-glass-clipart-1.jpg")
            
            drinkByName[name] = drink

['\ufeffname', 'category', 'measurement-1', 'ingredient-1', 'measurement-2', 'ingredient-2', 'measurement-3', 'ingredient-3', 'measurement-4', 'ingredient-4', 'measurement-5', 'ingredient-5', 'measurement-6', 'ingredient-6', 'instructions', 'glass', 'glass-size']


In [6]:
with open('all_drinks.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    line_count = 0
    
    for row in reader:
        line_count += 1
        if line_count == 1:
            print(row)
            
        else:
            
            name = row[1]
            
            ingredients = list()
            ing = (row[9], row[25])
            ingredients.append(ing)
            for i in range(7,15):
                ing = (row[9+i], row[25+i])
                ingredients.append(ing)
            for i in range(1, 7):
                ing = (row[9+i], row[25+i])
                ingredients.append(ing)
                
            src = row[6]

            drink = dict()
            drink['name'] = name
            drink['ingredients'] = ingredients
            drink['instructions'] = row[24]
            drink['src'] = src
            
            drink['reviews'] = []
            drink['rating'] = -1
            
            #drink = DrinkInfo(name=name, ingredients=ingredients, instructions=row[24],
            #                 src = src)
            
            drinkByName[name] = drink
            
        

['', 'strDrink', 'dateModified', 'idDrink', 'strAlcoholic', 'strCategory', 'strDrinkThumb', 'strGlass', 'strIBA', 'strIngredient1', 'strIngredient10', 'strIngredient11', 'strIngredient12', 'strIngredient13', 'strIngredient14', 'strIngredient15', 'strIngredient2', 'strIngredient3', 'strIngredient4', 'strIngredient5', 'strIngredient6', 'strIngredient7', 'strIngredient8', 'strIngredient9', 'strInstructions', 'strMeasure1', 'strMeasure10', 'strMeasure11', 'strMeasure12', 'strMeasure13', 'strMeasure14', 'strMeasure15', 'strMeasure2', 'strMeasure3', 'strMeasure4', 'strMeasure5', 'strMeasure6', 'strMeasure7', 'strMeasure8', 'strMeasure9', 'strVideo']


In [7]:
'''
import glob, os
from bs4 import BeautifulSoup

path = './ratings'

for filename in glob.glob(os.path.join(path, '*.html')):
    file = open(filename, 'r')
    
    html_doc = file.read()

    soup = BeautifulSoup(html_doc, 'html.parser')

    #for x in soup.find_all('div'): print(x)
    for r in soup.find_all('p', attrs={'itemprop':'reviewBody'}):
        
        print(r)
        name = r.itemprop.strip() # strip() is used to remove starting and trailing
        #print(name)
    '''

"\nimport glob, os\nfrom bs4 import BeautifulSoup\n\npath = './ratings'\n\nfor filename in glob.glob(os.path.join(path, '*.html')):\n    file = open(filename, 'r')\n    \n    html_doc = file.read()\n\n    soup = BeautifulSoup(html_doc, 'html.parser')\n\n    #for x in soup.find_all('div'): print(x)\n    for r in soup.find_all('p', attrs={'itemprop':'reviewBody'}):\n        \n        print(r)\n        name = r.itemprop.strip() # strip() is used to remove starting and trailing\n        #print(name)\n    "

In [14]:
## import glob, os
from urllib.request import urlopen
from bs4 import BeautifulSoup

def drink_scraper(url):
    page = urlopen(url)

    soup = BeautifulSoup(page, 'html.parser')
    
    drink = dict()
    
    name = soup.find('h1', attrs={'id':"recipe-main-content"})
    drink['name'] = name.text.strip()
    
    photo = soup.find('img', attrs={'class':'rec-photo'})
    drink['src'] = photo['src']
    
    drink['ingredients'] = list()
    
    for i in soup.find_all('span', attrs={'itemprop':'recipeIngredient'}):
        drink['ingredients'].append(i.text.strip())
        
    instr = list()
    
    for d in soup.find_all('span', attrs={'class':'recipe-directions__list--item'}):
        instr.append(d.text.strip())
        
    drink['instructions'] = '\n'.join(instr)
    
    staravg = soup.find('div', attrs={'class':'rating-stars'})
    
    drink['rating']= staravg['data-ratingstars']
    
    reviews = list()
    
        #for x in soup.find_all('div'): print(x)
    for review in soup.find_all('div', attrs={'class':'review-container clearfix'}):
        
        rating = dict()
        
        stars = review.find('span', attrs={'onclick':"AnchorScroll('reviews')"})
        
        rating['stars'] = stars['data-ratingstars']
        
        body = review.find('p', attrs={'itemprop':'reviewBody'})
        rating['body'] = body.text.strip()
        
        reviews.append(rating)
        
    drink['reviews'] = reviews
    
    return drink
    
    # For now, this version takes precedence
    #if drink['name'] in drinkByName:
        #print(drinkByName[drink['name']])
        #drinkByName[drink['name']]['reviews'] = drink['reviews']
        #drinkByName[drink['name']]['rating'] = drink['rating']
    #drinkByName[drink['name']] = drink

In [20]:
#scrape page 1 of rum drinks
import random, time

def scrape_category_page(page):
    soup = BeautifulSoup(page, 'html.parser')
    
    drinks = list()
        
    for drinkpage in soup.find_all('article', attrs={'class':'fixed-recipe-card'}):
        a = drinkpage.find('div', attrs={'class':'grid-card-image-container'})
        href = a.find('a', href=True)
        drink = drink_scraper(href['href'])
        print(drink['name'])
        drinkByName[drink['name']] = drink
        
        time.sleep(5 + random.randint(0,8))
        
def scrape_category():
    i = 1
    while True:
        #try:
            p = urlopen("https://www.allrecipes.com/recipes/1741/drinks/cocktails/?page=" + str(i))
            scrape_category_page(p)
            i += 1
            print(i)
        #except NameError:
            #print("DONE!")
            #return

#https://www.allrecipes.com/recipes/133/drinks/cocktails/
scrape_category()

Brenda's Strawberry Slush Delight
Mai Tai
Mojito Perfecto
Caribbean Rum Punch
Pina Colada III
Frozen Lime Daiquiri
Dark 'n' Stormy Cocktail
Absolute Stress
Shaggy's Hana Bay Frootie Joy
Amazingly Good Eggnog
Classic Spanish Sangria
The Real Mojito
Hot Buttered Rum Batter
Coquito
Bikini Martini
Green Grog
Bahama Mama
Best Strawberry Daiquiri
Colonial Hot Buttered Rum
Hot Spiked Cider
Christmas Creamy Eggnog
The Perfect Mai Tai
Pina Colada Punch
Rum-Spiked Horchata
Mrs. Baxton's Long Island Iced Tea
Mojitos by the Pitcher
2
Hurricane I
Single Cup Hot Buttered Rum
Luscious Eggnog
Mojito
Christmas Punch
Strawberry Mojito
The REAL Long Island Iced Tea
Caribbean Rum Punch
Tanning Bed Drink
Malibu Bay Breeze
Frozen Mojito
Puerto Rican Coquito
Mai Tai II
Sheila's Perfect Sangria
Hummingbird
Eggnog from Scratch
Blue Hawaiian Cocktail
Peppermint Eggnog Punch
Eggnog Extreme
Cranberry Rum Punch
3
Holiday Milk Punch: the New Eggnog
Danny's Warm and Fuzzy Apple Cider
Cuban Mojito
Eggnog Blend
Cherry

HTTPError: HTTP Error 404: Not Found

In [22]:
print(drinkByName['Mai Tai'])
#p = urlopen("https://www.google.com/")

{'reviews': [{'body': 'This drink is awesome...takes you right to the islands!  I agree with other viewers to use equal parts of spiced rum and coconut rum.  I made enough for 2 so I used 2 jiggers of each and a 6 oz ...', 'stars': '5'}, {'body': 'Ok, this drink is ALL wrong.  Where do I begin? First of all, NEVER used flavored or spice rum in Tiki drinks.  Use an aged rum and a white rum.  NEVER EVER USE MALIBU! Second, there is never or...', 'stars': '1'}, {'body': 'This drink is awesome...takes you right to the islands!  I agree with other viewers to use equal parts of spiced rum and coconut rum.  I made enough for 2 so I used 2 jiggers of each and a 6 oz ...', 'stars': '5'}, {'body': 'This was delicious and easy to try being 1 serving. I had everything on hand except spiced rum, so I used Myers dark rum. It was worth making another.', 'stars': '5'}, {'body': "Great drink! I made exactly as recipe stated. The two rums are what make this drink. I made for our memorial weekend BBQ - b

In [24]:
jsondata = {}
jsondata['drinks'] = []
for drink in drinkByName.values():
    try:
        ing = [(b +" " + a) for (a,b) in drink['ingredients'] if a != ""]
    except ValueError:
        ing = [a for a in drink['ingredients'] if a != ""]
        
    jsondata['drinks'].append({
        'name' : drink['name'],
        'description' : drink['instructions'],
        'src' : drink['src'],
        'ingredients' : ing,
        'rating' : drink['rating'],
        'reviews' : drink['reviews']
    })

In [25]:
with open('../app/static/drinks.json', 'w') as outfile:  
    json.dump(jsondata, outfile)